# Model Processing

## Imports & General Settings 

In [1]:
import unittest
import os
import sys
import pathlib
import urllib
import shutil
import re
import zipfile
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from IPython.display import display
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import wfdb
import pycwt as wavelet
from data import WaveletTransform, AFECGDataset
from PIL import Image
import dsp
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [3]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [4]:
images_per_sample = 20
total_data_size = dataset.get_len()
print("Total data size: ", total_data_size)

Total data size:  1397


In [5]:
data = [dataset[i][0] for i in range(total_data_size)]
labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [6]:
# samples, label = dataset[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [7]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), size=(256, 256))
# image_test = to_wavelet(data[0][0])
# transforms.ToPILImage()(test_img.permute(2, 1, 0)).show()

##  Wavelet Transform

In [8]:
# Total data size is 1397
# You can choose the data size 
data_size = 5

In [9]:
fmt = '../data/images/sample_{}_win_{}.pt'
# to_wavelet = WaveletTransform(wavelet.Morlet(6), size=(256, 256))
# start = time.time()

# skip = 0
# for sample_idx in range(data_size):
#     sample = data[sample_idx]
#     for signal_idx, signal in enumerate(sample):
#         filepath = fmt.format(sample_idx, signal_idx)
#         if os.path.isfile(filepath):
#              # print('Skip {},{}'.format(sample_idx, signal_idx))
#             skip += 1
#             continue
#         new_sample = to_wavelet(signal)
#         torch.save(new_sample, filepath)
    
# end = time.time()
# print('Elapsed time: {} ms'.format(1000 * (end - start)))
# print('Skipped {} files'.format(skip))

In [10]:
transformed_data = []
transformed_labels= []

for sample_idx in range(data_size):
    new_sample = []
    for signal_idx in range(20):
        img = torch.load(fmt.format(sample_idx, signal_idx))
        new_sample.append(img)
    transformed_data.append(new_sample)
    transformed_labels.append(labels[sample_idx])

## Train & Test set creation

In [11]:
x_train, x_test, y_train, y_test =  train_test_split(transformed_data, transformed_labels, test_size=0.2, random_state=1)

## CNN

In [12]:
class ConvNet(nn.Module):
    def __init__(self, in_channels=3):
        super(ConvNet, self).__init__()
                
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 10, kernel_size=(3,21)),
            nn.ReLU(),
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(3,21)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(10, 10, kernel_size=(4,21)),
            nn.ReLU(),
        )
        
        self.fc = nn.Linear(81600, 50)
        
    def forward(self, x):
        # print(x.shape)
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)

        # out = out.reshape(out.size(0), -1)

        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        
        # print(out.shape)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        # print(out.shape)
        return out

In [13]:
display(ConvNet(in_channels=3))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Linear(in_features=81600, out_features=50, bias=True)
)

In [14]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet()

# h = encoder_cnn(x0.permute(2, 0, 1).unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

In [15]:
# model = ConvNet()
# num_epochs = 100
# total_size = len(x_train)
# test.assertEqual(total_size, len(y_train))

# # Loss and optimizer
# learning_rate = 0.001
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Train the model
# total_step = len(x_train)
# loss_list = []
# acc_list = []

# for epoch in range(num_epochs):
#     acc = 0
#     for idx, (samples, label) in enumerate(zip(x_train, y_train)):
#         label = torch.tensor([label]).long()
#         for image in samples:
            
#             # Run the forward pass
#             output = model(image.float().permute(2, 0, 1).unsqueeze(0))
#             loss = criterion(output, label)
#             loss_list.append(loss.item())
            
#             # Backprop and perform Adam optimisation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             # Track the accuracy
#             _, predicted = torch.max(output.data, 1)
#             correct = (predicted == label).sum().item()
#             acc += correct
                        
#     acc = acc / (total_size * 20)          
#     acc_list.append(acc)
#     print('Epoch [{}/{}], Accuracy: {:.2f}%'
#           .format(epoch + 1, num_epochs, acc * 100))    

## BRNN

In [94]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_gates):
        super(BRNN, self).__init__()
        self.bi_grus = torch.nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_gates,
                                    batch_first=False, bidirectional=True)

    def forward(self, X):
        output, hn = self.bi_grus(X)
        return output

In [95]:
display(BRNN(50, 50, images_per_sample))

BRNN(
  (bi_grus): GRU(50, 50, num_layers=20, bidirectional=True)
)

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [143]:
class SoftmaxAttention(nn.Module):
    def __init__(self, input_size):
        super(SoftmaxAttention, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(1, input_size))
        torch.nn.init.xavier_uniform_(self.weight)
    
    def forward(self, X):
        # X [T x N]
        # weight [N x 1]
        print('X: ', X)
        alignment_scores = X.matmul(self.weight.t())
        print('AS: ', alignment_scores)
        # print('wT: ', self.weight.t().shape)
        # print('AS: ', alignment_scores.shape)
        # alpha [T x 1]
        attn_weights = nn.functional.softmax(alignment_scores, dim=1)
        print('ATT: ', attn_weights)
        # print('X: ', X.shape)
        # h_att [1 x N]
        return torch.matmul(attn_weights.transpose(0, 1), X)

## Full model

In [145]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        
        lst = []
        for i in range(images_per_sample):
            
            lst.append(ConvNet())
            self.add_module('conv{}'.format(i), lst[i])
            
        self.cnn_layers = lst
        self.brnn = BRNN(50, 50, images_per_sample)
        self.attention = SoftmaxAttention(100)
        self.fc = nn.Linear(100, 2)


    def forward(self, X):        
        out = [self.cnn_layers[i](X[i].float().permute(2, 0, 1).unsqueeze(0)) for i in range(images_per_sample)]
        out = torch.cat(out).unsqueeze(1)
        # print('After CNN: ', out)
        out = self.brnn(out)
        # print('After BRNN: ', out)
        out = self.attention(out.squeeze(1))
        # print('After ATT: ', out)
        out = self.fc(out)
        # print('After FC: ', out)
        return out

In [141]:
# display(Baseline())

### Training 

In [ ]:
model = Baseline()
num_epochs = 5
total_size = len(x_train)
test.assertEqual(total_size, len(y_train))

# Loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = total_size
loss_list = []
acc_list = []

for epoch in range(num_epochs):
    acc = 0
    for idx, (samples, label) in enumerate(zip(x_train, y_train)):
        label = torch.tensor([label]).long()

        # Run the forward pass
        output = model(samples)
        print(output, label)
        loss = criterion(output, label)
        loss_list.append(loss.item())
        
        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        # Track the accuracy
        prediction = nn.functional.softmax(output, dim=0).argmax(dim=0)
        correct = (prediction == label).sum().item()
        acc += correct
                    
    acc = acc / (total_size)          
    acc_list.append(acc)
    print('Epoch [{}/{}], Accuracy: {:.2f}%'
          .format(epoch + 1, num_epochs, acc * 100))    

X:  tensor([[ 0.1308, -0.0136, -0.0275,  ..., -0.1416, -0.1041, -0.0101],
        [ 0.1920, -0.0251, -0.0490,  ..., -0.1475, -0.0886, -0.0065],
        [ 0.2208, -0.0303, -0.0652,  ..., -0.1499, -0.0769, -0.0043],
        ...,
        [ 0.2620, -0.0344, -0.0927,  ..., -0.0459,  0.0037,  0.0181],
        [ 0.2445, -0.0479, -0.0944,  ..., -0.0157,  0.0058,  0.0228],
        [ 0.2149, -0.0623, -0.1005,  ...,  0.0065,  0.0056,  0.0230]],
       grad_fn=<SqueezeBackward1>)
AS:  tensor([[-0.1875],
        [-0.2037],
        [-0.2230],
        [-0.2416],
        [-0.2569],
        [-0.2678],
        [-0.2742],
        [-0.2769],
        [-0.2765],
        [-0.2738],
        [-0.2691],
        [-0.2629],
        [-0.2554],
        [-0.2470],
        [-0.2380],
        [-0.2286],
        [-0.2180],
        [-0.2036],
        [-0.1792],
        [-0.1319]], grad_fn=<MmBackward>)
ATT:  tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
 

# 